In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.6 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from haversine import haversine

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor, Pool, cv
from sklearn.metrics import mean_absolute_error

In [5]:
df_answer_sample=pd.read_csv('/content/drive/MyDrive/dataset/answer_sample.csv',sep=',',encoding='utf-8')

df_train_GJ=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/공주.csv',sep=',',encoding='utf-8')
df_train_NE=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/노은동.csv',sep=',',encoding='utf-8')
df_train_NS=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/논산.csv',sep=',',encoding='utf-8')
df_train_DC=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/대천2동.csv',sep=',',encoding='utf-8')
df_train_DG=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/독곶리.csv',sep=',',encoding='utf-8')
df_train_DM=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/동문동.csv',sep=',',encoding='utf-8')
df_train_MJ=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/모종동.csv',sep=',',encoding='utf-8')
df_train_MC=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/문창동.csv',sep=',',encoding='utf-8')
df_train_SS=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/성성동.csv',sep=',',encoding='utf-8')
df_train_SB=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/신방동.csv',sep=',',encoding='utf-8')
df_train_SH=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/신흥동.csv',sep=',',encoding='utf-8')
df_train_AR=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/아름동.csv',sep=',',encoding='utf-8')
df_train_YS=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/예산군.csv',sep=',',encoding='utf-8')
df_train_EN=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/읍내동.csv',sep=',',encoding='utf-8')
df_train_YO=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/이원면.csv',sep=',',encoding='utf-8')
df_train_JL=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/정림동.csv',sep=',',encoding='utf-8')
df_train_HS=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN/홍성읍.csv',sep=',',encoding='utf-8')

df_train=df_train_GJ.append(df_train_NE)
df_train=df_train.append(df_train_NS)
df_train=df_train.append(df_train_DC)
df_train=df_train.append(df_train_DG)
df_train=df_train.append(df_train_DM)
df_train=df_train.append(df_train_MJ)
df_train=df_train.append(df_train_MC)
df_train=df_train.append(df_train_SS)
df_train=df_train.append(df_train_SB)
df_train=df_train.append(df_train_SH)
df_train=df_train.append(df_train_AR)
df_train=df_train.append(df_train_YS)
df_train=df_train.append(df_train_EN)
df_train=df_train.append(df_train_YO)
df_train=df_train.append(df_train_JL)
df_train=df_train.append(df_train_HS)

df_pmmap=pd.read_csv('/content/drive/MyDrive/dataset/META/pmmap.csv',sep=',',encoding='utf-8')
df_awsmap=pd.read_csv('/content/drive/MyDrive/dataset/META/awsmap.csv',sep=',',encoding='utf-8')

df_trainaws_GR=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/계룡.csv',sep=',',encoding='utf-8')
df_trainaws_GJ=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/공주.csv',sep=',',encoding='utf-8')
df_trainaws_NS=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/논산.csv',sep=',',encoding='utf-8')
df_trainaws_DJ=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/당진.csv',sep=',',encoding='utf-8')
df_trainaws_DS=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/대산.csv',sep=',',encoding='utf-8')
df_trainaws_DC=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/대천항.csv',sep=',',encoding='utf-8')
df_trainaws_BG=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/북격렬비도.csv',sep=',',encoding='utf-8')
df_trainaws_SC=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/서천.csv',sep=',',encoding='utf-8')
df_trainaws_SG=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/성거.csv',sep=',',encoding='utf-8')
df_trainaws_SJGE=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/세종고운.csv',sep=',',encoding='utf-8')
df_trainaws_SJGN=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/세종금남.csv',sep=',',encoding='utf-8')
df_trainaws_SGYS=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/세종연서.csv',sep=',',encoding='utf-8')
df_trainaws_SGJE=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/세종전의.csv',sep=',',encoding='utf-8')
df_trainaws_SC2=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/세천.csv',sep=',',encoding='utf-8')
df_trainaws_AS=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/아산.csv',sep=',',encoding='utf-8')
df_trainaws_AD=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/안도.csv',sep=',',encoding='utf-8')
df_trainaws_YH=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/양화.csv',sep=',',encoding='utf-8')
df_trainaws_YS=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/예산.csv',sep=',',encoding='utf-8')
df_trainaws_OW=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/오월드.csv',sep=',',encoding='utf-8')
df_trainaws_OD=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/옹도.csv',sep=',',encoding='utf-8')
df_trainaws_UG=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/유구.csv',sep=',',encoding='utf-8')
df_trainaws_JD=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/장동.csv',sep=',',encoding='utf-8')
df_trainaws_JS=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/정산.csv',sep=',',encoding='utf-8')
df_trainaws_JA=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/정안.csv',sep=',',encoding='utf-8')
df_trainaws_CY=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/청양.csv',sep=',',encoding='utf-8')
df_trainaws_CJ=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/춘장대.csv',sep=',',encoding='utf-8')
df_trainaws_TA=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/태안.csv',sep=',',encoding='utf-8')
df_trainaws_HD=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/호도.csv',sep=',',encoding='utf-8')
df_trainaws_HB=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/홍북.csv',sep=',',encoding='utf-8')
df_trainaws_HS=pd.read_csv('/content/drive/MyDrive/dataset/TRAIN_AWS/홍성죽도.csv',sep=',',encoding='utf-8')

list_trainaws=[df_trainaws_GR,
               df_trainaws_GJ,
               df_trainaws_NS,
               df_trainaws_DJ,
               df_trainaws_DS,
               df_trainaws_DC,
               df_trainaws_BG,
               df_trainaws_SC,
               df_trainaws_SG,
               df_trainaws_SJGE,
               df_trainaws_SJGN,
               df_trainaws_SGYS,
               df_trainaws_SGJE,
               df_trainaws_SC2,
               df_trainaws_AS,
               df_trainaws_AD,
               df_trainaws_YH,
               df_trainaws_YS,
               df_trainaws_OW,
               df_trainaws_OD,
               df_trainaws_UG,
               df_trainaws_JD,
               df_trainaws_JS,
               df_trainaws_JA,
               df_trainaws_CY,
               df_trainaws_CJ,
               df_trainaws_TA,
               df_trainaws_HD,
               df_trainaws_HB,
               df_trainaws_HS]

df_trainaws=df_trainaws_GR.append(df_trainaws_GJ)
for i in range(28):
  df_trainaws=df_trainaws.append(list_trainaws[i+2])

df_testaws_GR=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/계룡.csv',sep=',',encoding='utf-8')
df_testaws_GJ=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/공주.csv',sep=',',encoding='utf-8')
df_testaws_NS=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/논산.csv',sep=',',encoding='utf-8')
df_testaws_DJ=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/당진.csv',sep=',',encoding='utf-8')
df_testaws_DS=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/대산.csv',sep=',',encoding='utf-8')
df_testaws_DC=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/대천항.csv',sep=',',encoding='utf-8')
df_testaws_BG=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/북격렬비도.csv',sep=',',encoding='utf-8')
df_testaws_SC=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/서천.csv',sep=',',encoding='utf-8')
df_testaws_SG=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/성거.csv',sep=',',encoding='utf-8')
df_testaws_SJGE=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/세종고운.csv',sep=',',encoding='utf-8')
df_testaws_SJGN=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/세종금남.csv',sep=',',encoding='utf-8')
df_testaws_SGYS=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/세종연서.csv',sep=',',encoding='utf-8')
df_testaws_SGJE=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/세종전의.csv',sep=',',encoding='utf-8')
df_testaws_SC2=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/세천.csv',sep=',',encoding='utf-8')
df_testaws_AS=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/아산.csv',sep=',',encoding='utf-8')
df_testaws_AD=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/안도.csv',sep=',',encoding='utf-8')
df_testaws_YH=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/양화.csv',sep=',',encoding='utf-8')
df_testaws_YS=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/예산.csv',sep=',',encoding='utf-8')
df_testaws_OW=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/오월드.csv',sep=',',encoding='utf-8')
df_testaws_OD=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/옹도.csv',sep=',',encoding='utf-8')
df_testaws_UG=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/유구.csv',sep=',',encoding='utf-8')
df_testaws_JD=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/장동.csv',sep=',',encoding='utf-8')
df_testaws_JS=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/정산.csv',sep=',',encoding='utf-8')
df_testaws_JA=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/정안.csv',sep=',',encoding='utf-8')
df_testaws_CY=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/청양.csv',sep=',',encoding='utf-8')
df_testaws_CJ=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/춘장대.csv',sep=',',encoding='utf-8')
df_testaws_TA=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/태안.csv',sep=',',encoding='utf-8')
df_testaws_HD=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/호도.csv',sep=',',encoding='utf-8')
df_testaws_HB=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/홍북.csv',sep=',',encoding='utf-8')
df_testaws_HS=pd.read_csv('/content/drive/MyDrive/dataset/TEST_AWS/홍성죽도.csv',sep=',',encoding='utf-8')

list_testaws=[df_testaws_GR,
               df_testaws_GJ,
               df_testaws_NS,
               df_testaws_DJ,
               df_testaws_DS,
               df_testaws_DC,
               df_testaws_BG,
               df_testaws_SC,
               df_testaws_SG,
               df_testaws_SJGE,
               df_testaws_SJGN,
               df_testaws_SGYS,
               df_testaws_SGJE,
               df_testaws_SC2,
               df_testaws_AS,
               df_testaws_AD,
               df_testaws_YH,
               df_testaws_YS,
               df_testaws_OW,
               df_testaws_OD,
               df_testaws_UG,
               df_testaws_JD,
               df_testaws_JS,
               df_testaws_JA,
               df_testaws_CY,
               df_testaws_CJ,
               df_testaws_TA,
               df_testaws_HD,
               df_testaws_HB,
               df_testaws_HS]

df_testaws=df_testaws_GR.append(df_testaws_GJ)
for i in range(28):
  df_testaws=df_testaws.append(list_testaws[i+2])

<ipython-input-5-6cfb9457092f>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train=df_train_GJ.append(df_train_NE)
<ipython-input-5-6cfb9457092f>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train=df_train.append(df_train_NS)
<ipython-input-5-6cfb9457092f>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train=df_train.append(df_train_DC)
<ipython-input-5-6cfb9457092f>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train=df_train.append(df_train_DG)
<ipython-input-5-6cfb9457092f>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instea

In [6]:
df_test_GJ=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/공주.csv',sep=',',encoding='utf-8')
df_test_NE=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/노은동.csv',sep=',',encoding='utf-8')
df_test_NS=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/논산.csv',sep=',',encoding='utf-8')
df_test_DC=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/대천2동.csv',sep=',',encoding='utf-8')
df_test_DG=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/독곶리.csv',sep=',',encoding='utf-8')
df_test_DM=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/동문동.csv',sep=',',encoding='utf-8')
df_test_MJ=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/모종동.csv',sep=',',encoding='utf-8')
df_test_MC=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/문창동.csv',sep=',',encoding='utf-8')
df_test_SS=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/성성동.csv',sep=',',encoding='utf-8')
df_test_SB=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/신방동.csv',sep=',',encoding='utf-8')
df_test_SH=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/신흥동.csv',sep=',',encoding='utf-8')
df_test_AR=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/아름동.csv',sep=',',encoding='utf-8')
df_test_YS=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/예산군.csv',sep=',',encoding='utf-8')
df_test_EN=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/읍내동.csv',sep=',',encoding='utf-8')
df_test_YO=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/이원면.csv',sep=',',encoding='utf-8')
df_test_JL=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/정림동.csv',sep=',',encoding='utf-8')
df_test_HS=pd.read_csv('/content/drive/MyDrive/dataset/TEST_INPUT/홍성읍.csv',sep=',',encoding='utf-8')

df_test=df_test_GJ.append(df_test_NE)
df_test=df_test.append(df_test_NS)
df_test=df_test.append(df_test_DC)
df_test=df_test.append(df_test_DG)
df_test=df_test.append(df_test_DM)
df_test=df_test.append(df_test_MJ)
df_test=df_test.append(df_test_MC)
df_test=df_test.append(df_test_SS)
df_test=df_test.append(df_test_SB)
df_test=df_test.append(df_test_SH)
df_test=df_test.append(df_test_AR)
df_test=df_test.append(df_test_YS)
df_test=df_test.append(df_test_EN)
df_test=df_test.append(df_test_YO)
df_test=df_test.append(df_test_JL)
df_test=df_test.append(df_test_HS)

<ipython-input-6-32f57b4989e6>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test=df_test_GJ.append(df_test_NE)
<ipython-input-6-32f57b4989e6>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test=df_test.append(df_test_NS)
<ipython-input-6-32f57b4989e6>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test=df_test.append(df_test_DC)
<ipython-input-6-32f57b4989e6>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test=df_test.append(df_test_DG)
<ipython-input-6-32f57b4989e6>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test

In [7]:
df_nearest=df_pmmap.iloc[:,0:3]

nearest=[]

for i in range(df_nearest.shape[0]):

  dist=[0 for j in range(df_awsmap.shape[0])]

  for j in range(df_awsmap.shape[0]):
    pm=(df_nearest.iloc[i,1],df_nearest.iloc[i,2])
    aws=(df_awsmap.iloc[j,1],df_awsmap.iloc[j,2])
    h=haversine(pm,aws,unit='m')
    dist[j]=h

  index=np.argmin(dist)
  nearest.append(df_awsmap.iloc[index,0])

df_nearest['NearestAms']=nearest
df_nearest=df_nearest.iloc[:,[0,3]]

In [8]:
df_nearest

,Location,NearestAms
0,아름동,세종고운
1,신흥동,세종연서
2,노은동,계룡
3,문창동,오월드
4,읍내동,장동
5,정림동,오월드
6,공주,공주
7,논산,논산
8,대천2동,대천항
9,독곶리,대산


In [9]:
df_train_new=pd.merge(df_train,df_nearest,left_on='측정소',right_on='Location',how='left')
df_train_new=pd.merge(df_train_new,df_trainaws,left_on=['연도','일시','NearestAms'],right_on=['연도','일시','지점'],how='left')

df_train_new['일시']=pd.to_datetime(df_train_new['일시'],format='%m-%d %H:%M',errors='coerce')

df_train_new['Month']=df_train_new['일시'].dt.month
df_train_new['Day']=df_train_new['일시'].dt.day
df_train_new['Hour']=df_train_new['일시'].dt.hour

df_train_new.drop(['일시'], axis=1, inplace=True)

In [10]:
df_train_new

,연도,측정소,PM2.5,Location,NearestAms,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),Month,Day,Hour
0,0,공주,0.056,공주,공주,공주,0.173776,0.201944,0.023018,0.0,0.828,1.0,1.0,0.0
1,0,공주,0.060,공주,공주,공주,0.176935,0.168611,0.030691,0.0,0.831,1.0,1.0,1.0
2,0,공주,0.068,공주,공주,공주,0.180095,0.087222,0.033248,0.0,0.784,1.0,1.0,2.0
3,0,공주,0.060,공주,공주,공주,0.178515,0.087222,0.025575,0.0,0.745,1.0,1.0,3.0
4,0,공주,0.068,공주,공주,공주,0.164297,0.113889,0.020460,0.0,0.750,1.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596083,3,홍성읍,0.060,홍성읍,홍북,홍북,0.273302,0.832222,0.086957,0.0,0.671,12.0,31.0,19.0
596084,3,홍성읍,0.052,홍성읍,홍북,홍북,0.271722,0.831667,0.043478,0.0,0.692,12.0,31.0,20.0
596085,3,홍성읍,0.044,홍성읍,홍북,홍북,0.268562,0.832500,0.066496,0.0,0.706,12.0,31.0,21.0
596086,3,홍성읍,0.052,홍성읍,홍북,홍북,0.262243,0.866944,0.043478,0.0,0.725,12.0,31.0,22.0


In [11]:
df_train_new.drop(['측정소', '지점'], axis=1, inplace=True)
df_train_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596088 entries, 0 to 596087
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   연도          596088 non-null  int64  
 1   PM2.5       580546 non-null  float64
 2   Location    596088 non-null  object 
 3   NearestAms  596088 non-null  object 
 4   기온(°C)      585790 non-null  float64
 5   풍향(deg)     585790 non-null  float64
 6   풍속(m/s)     585790 non-null  float64
 7   강수량(mm)     585790 non-null  float64
 8   습도(%)       585790 non-null  float64
 9   Month       595680 non-null  float64
 10  Day         595680 non-null  float64
 11  Hour        595680 non-null  float64
dtypes: float64(9), int64(1), object(2)
memory usage: 59.1+ MB


In [12]:
ohe=OneHotEncoder(sparse=False)
df_onehot=pd.DataFrame(ohe.fit_transform(df_train_new[['Location','NearestAms']]))
df_train_new=pd.concat([df_train_new,df_onehot],axis=1)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
df_train_new.drop(['Location','NearestAms'],axis=1, inplace=True)

In [14]:
df_train_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596088 entries, 0 to 596087
Data columns (total 41 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   연도       596088 non-null  int64  
 1   PM2.5    580546 non-null  float64
 2   기온(°C)   585790 non-null  float64
 3   풍향(deg)  585790 non-null  float64
 4   풍속(m/s)  585790 non-null  float64
 5   강수량(mm)  585790 non-null  float64
 6   습도(%)    585790 non-null  float64
 7   Month    595680 non-null  float64
 8   Day      595680 non-null  float64
 9   Hour     595680 non-null  float64
 10  0        596088 non-null  float64
 11  1        596088 non-null  float64
 12  2        596088 non-null  float64
 13  3        596088 non-null  float64
 14  4        596088 non-null  float64
 15  5        596088 non-null  float64
 16  6        596088 non-null  float64
 17  7        596088 non-null  float64
 18  8        596088 non-null  float64
 19  9        596088 non-null  float64
 20  10       596088 non-null  

In [15]:
df_test_new=pd.merge(df_test,df_nearest,left_on='측정소',right_on='Location',how='left')

df_testaws_mean=pd.DataFrame(df_trainaws.groupby(['지점','일시']).mean())
df_testaws_mean=df_testaws_mean.reset_index()
df_testaws_mean=df_testaws_mean.drop(['연도'],axis=1)

df_test_new=pd.merge(df_test_new,df_testaws_mean,left_on=['일시','NearestAms'],right_on=['일시','지점'],how='left')

df_test_new['일시']=pd.to_datetime(df_test_new['일시'],format='%m-%d %H:%M',errors='coerce')
df_test_new['Month']=df_test_new['일시'].dt.month
df_test_new['Day']=df_test_new['일시'].dt.day
df_test_new['Hour']=df_test_new['일시'].dt.hour

df_test_new=df_test_new.drop(['일시'],axis=1)

In [16]:
df_test_new.drop(['측정소', '지점'], axis=1, inplace=True)
df_test_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131376 entries, 0 to 131375
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   연도          131376 non-null  int64  
 1   PM2.5       53040 non-null   float64
 2   Location    131376 non-null  object 
 3   NearestAms  131376 non-null  object 
 4   기온(°C)      131376 non-null  float64
 5   풍향(deg)     131376 non-null  float64
 6   풍속(m/s)     131376 non-null  float64
 7   강수량(mm)     131376 non-null  float64
 8   습도(%)       131376 non-null  float64
 9   Month       131376 non-null  int64  
 10  Day         131376 non-null  int64  
 11  Hour        131376 non-null  int64  
dtypes: float64(6), int64(4), object(2)
memory usage: 13.0+ MB


In [17]:
df_test_new

,연도,PM2.5,Location,NearestAms,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),Month,Day,Hour
0,4,0.060,공주,공주,0.286730,0.077917,0.011509,0.0,0.75950,1,1,0
1,4,0.064,공주,공주,0.281596,0.284931,0.015985,0.0,0.77800,1,1,1
2,4,0.072,공주,공주,0.282780,0.191944,0.017263,0.0,0.75525,1,1,2
3,4,0.064,공주,공주,0.280016,0.208403,0.012788,0.0,0.79025,1,1,3
4,4,0.056,공주,공주,0.267773,0.110347,0.012148,0.0,0.81850,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
131371,4,0.056,홍성읍,홍북,0.434439,0.545208,0.019821,0.0,0.63300,11,18,19
131372,4,0.036,홍성읍,홍북,0.420221,0.291250,0.015345,0.0,0.66900,11,18,20
131373,4,0.020,홍성읍,홍북,0.414692,0.361736,0.021100,0.0,0.68325,11,18,21
131374,4,0.024,홍성읍,홍북,0.406003,0.467222,0.024936,0.0,0.71350,11,18,22


In [18]:
df_onehot2=pd.DataFrame(ohe.fit_transform(df_test_new[['Location','NearestAms']]))
df_test_new=pd.concat([df_test_new,df_onehot2],axis=1)
df_test_new.drop(['Location','NearestAms'],axis=1, inplace=True)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [19]:
df_test_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131376 entries, 0 to 131375
Data columns (total 41 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   연도       131376 non-null  int64  
 1   PM2.5    53040 non-null   float64
 2   기온(°C)   131376 non-null  float64
 3   풍향(deg)  131376 non-null  float64
 4   풍속(m/s)  131376 non-null  float64
 5   강수량(mm)  131376 non-null  float64
 6   습도(%)    131376 non-null  float64
 7   Month    131376 non-null  int64  
 8   Day      131376 non-null  int64  
 9   Hour     131376 non-null  int64  
 10  0        131376 non-null  float64
 11  1        131376 non-null  float64
 12  2        131376 non-null  float64
 13  3        131376 non-null  float64
 14  4        131376 non-null  float64
 15  5        131376 non-null  float64
 16  6        131376 non-null  float64
 17  7        131376 non-null  float64
 18  8        131376 non-null  float64
 19  9        131376 non-null  float64
 20  10       131376 non-null  

In [20]:
df_submit=pd.merge(df_answer_sample,df_nearest,left_on='측정소',right_on='Location',how='left')

df_submitaws_mean=pd.DataFrame(df_trainaws.groupby(['지점','일시']).mean())
df_submitaws_mean=df_submitaws_mean.reset_index()
df_submitaws_mean=df_submitaws_mean.drop(['연도'],axis=1)

df_submit=pd.merge(df_submit,df_submitaws_mean,left_on=['일시','NearestAms'],right_on=['일시','지점'],how='left')

df_submit=df_submit.iloc[:,[0,1,3,4,5,7,8,9,10,11]]

df_submit['일시']=pd.to_datetime(df_submit['일시'],format='%m-%d %H:%M',errors='coerce')
df_submit['Month']=df_submit['일시'].dt.month
df_submit['Day']=df_submit['일시'].dt.day
df_submit['Hour']=df_submit['일시'].dt.hour

df_submit=df_submit.drop(['일시'],axis=1)

df_onehot3=pd.DataFrame(ohe.fit_transform(df_submit[['Location','NearestAms']]))
df_submit=pd.concat([df_submit,df_onehot3],axis=1)
df_submit.drop(['Location','NearestAms'],axis=1, inplace=True)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [21]:
df_submit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78336 entries, 0 to 78335
Data columns (total 41 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   연도       78336 non-null  int64  
 1   PM2.5    0 non-null      float64
 2   기온(°C)   78336 non-null  float64
 3   풍향(deg)  78336 non-null  float64
 4   풍속(m/s)  78336 non-null  float64
 5   강수량(mm)  78336 non-null  float64
 6   습도(%)    78336 non-null  float64
 7   Month    78336 non-null  int64  
 8   Day      78336 non-null  int64  
 9   Hour     78336 non-null  int64  
 10  0        78336 non-null  float64
 11  1        78336 non-null  float64
 12  2        78336 non-null  float64
 13  3        78336 non-null  float64
 14  4        78336 non-null  float64
 15  5        78336 non-null  float64
 16  6        78336 non-null  float64
 17  7        78336 non-null  float64
 18  8        78336 non-null  float64
 19  9        78336 non-null  float64
 20  10       78336 non-null  float64
 21  11       783

In [22]:
df_train_new=df_train_new.dropna()
X_train=df_train_new.drop(['PM2.5'],axis=1)
X_train=X_train.values

y_train=df_train_new['PM2.5'].values

In [23]:
df_test_new=df_test_new.dropna()
X_test=df_test_new.drop(['PM2.5'],axis=1)
X_test=X_test.values

y_test=df_test_new['PM2.5'].values

In [24]:
df_submit=df_submit.drop(['PM2.5'],axis=1)
df_submit=df_submit.values

In [29]:
train_dataset = Pool(data=X_train, label=y_train)

cat=CatBoostRegressor(random_state=36, loss_function='MAE')

parameters = {'depth'         : [4,5,6,7,8],
                            'learning_rate' : [0.01,0.02,0.03,0.04,0.5],
                            'iterations'    : [10, 50, 90,125,150]
                        }

grid_cat = GridSearchCV(estimator=cat, param_grid = parameters, cv = 3, n_jobs=-1)
grid_cat.fit(X_train, y_train)

0:	learn: 0.0444258	total: 252ms	remaining: 37.5s
1:	learn: 0.0426777	total: 400ms	remaining: 29.6s
2:	learn: 0.0415717	total: 540ms	remaining: 26.5s
3:	learn: 0.0408819	total: 659ms	remaining: 24.1s
4:	learn: 0.0400748	total: 795ms	remaining: 23.1s
5:	learn: 0.0398068	total: 918ms	remaining: 22s
6:	learn: 0.0393417	total: 1.05s	remaining: 21.5s
7:	learn: 0.0389996	total: 1.17s	remaining: 20.7s
8:	learn: 0.0387207	total: 1.27s	remaining: 19.9s
9:	learn: 0.0385043	total: 1.39s	remaining: 19.4s
10:	learn: 0.0381247	total: 1.5s	remaining: 19s
11:	learn: 0.0379770	total: 1.61s	remaining: 18.6s
12:	learn: 0.0378390	total: 1.72s	remaining: 18.2s
13:	learn: 0.0374380	total: 1.84s	remaining: 17.9s
14:	learn: 0.0372916	total: 1.95s	remaining: 17.5s
15:	learn: 0.0371708	total: 2.06s	remaining: 17.3s
16:	learn: 0.0368383	total: 2.18s	remaining: 17.1s
17:	learn: 0.0361247	total: 2.29s	remaining: 16.8s
18:	learn: 0.0360190	total: 2.41s	remaining: 16.6s
19:	learn: 0.0359043	total: 2.56s	remaining: 1

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostRegressor object at 0x7fc751856440>,
             n_jobs=-1,
             param_grid={'depth': [4, 5, 6, 7, 8],
                         'iterations': [10, 50, 90, 125, 150],
                         'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.5]})

In [31]:
print("\n The best parameters across ALL searched params:\n",grid_cat.best_params_)


 The best parameters across ALL searched params:
 {'depth': 8, 'iterations': 150, 'learning_rate': 0.5}


In [32]:
y_train_pred=grid_cat.predict(train_dataset)
mean_absolute_error(y_train,y_train_pred)

0.029066316719661454

In [34]:
test_dataset = Pool(data=X_test, label=y_test)
y_test_pred=grid_cat.predict(test_dataset)
mean_absolute_error(y_test,y_test_pred)

0.04016555904350737

In [35]:
y_submit_pred = grid_cat.predict(df_submit)

In [36]:
df_answer_sample.shape, y_submit_pred.shape

((78336, 4), (78336,))

In [37]:
df_answer_sample['PM2.5']=y_submit_pred
df_answer_sample.to_csv('/content/drive/MyDrive/dataset/answer_sample230505_GridSearch.csv',encoding='utf-8',index=False)